In [3]:
import torch
import joblib
import pandas as pd
import torch.nn as nn
from sklearn.metrics import classification_report

In [4]:
# --- CONFIG ---
MODEL_PATH = 'model.pth'
PREPROCESSOR_PATH = 'preprocessor.pkl'
TEST_DATA_PATH = 'test_set_unseen.csv'
THRESHOLD = 0.15

In [5]:
class BankModel(nn.Module):
    def __init__(self, input_dim):
        super(BankModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 32)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        self.layer2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.output_layer = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.dropout1(self.relu1(self.layer1(x)))
        x = self.dropout2(self.relu2(self.layer2(x)))
        x = self.sigmoid(self.output_layer(x))
        return x

In [6]:
print("Loading system...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading system...


In [7]:
# Translator loading
preprocessor = joblib.load(PREPROCESSOR_PATH)

# Load the Brain
model = BankModel(input_dim=62) # We know it's 62 inputs
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# Load the Test Data
df_test = pd.read_csv(TEST_DATA_PATH)

# Separate the Answers (y) from the Questions (X)
y_actual = df_test['y']
X_raw = df_test.drop(columns=['y'])

# Run the Pipeline
# Preprocess (Scale/Encode)
X_processed = preprocessor.transform(X_raw)

# Convert to Tensor
X_tensor = torch.tensor(X_processed, dtype=torch.float32).to(device)

In [8]:
# Predict
print("Running predictions...")
with torch.no_grad():
    y_probs = model(X_tensor).cpu().numpy().flatten()

# Apply Decision Rule
y_pred = (y_probs > THRESHOLD).astype(int)

Running predictions...


In [9]:
# Show the Report
print(f"\n--- Final Report (Threshold: {THRESHOLD}) ---")
print(classification_report(y_actual, y_pred, target_names=['No', 'Yes']))


--- Final Report (Threshold: 0.15) ---
              precision    recall  f1-score   support

          No       0.95      0.90      0.92      7308
         Yes       0.44      0.61      0.51       928

    accuracy                           0.87      8236
   macro avg       0.69      0.76      0.72      8236
weighted avg       0.89      0.87      0.88      8236

